In [1]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START
from langgraph.graph.message import add_messages
from dotenv import load_dotenv
from IPython.display import Image, display
import gradio as gr
from langgraph.prebuilt import ToolNode, tools_condition
import requests
import os
from langchain_core.tools import Tool

from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver

In [2]:
load_dotenv(override=True)

True

In [3]:
class State(TypedDict):
    
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)

In [4]:
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_user = os.getenv("PUSHOVER_USER")
pushover_url = "https://api.pushover.net/1/messages.json"

def push(text: str):
    """Send a push notification to the user"""
    requests.post(pushover_url, data = {"token": pushover_token, "user": pushover_user, "message": text})

tool_push = Tool(
        name="send_push_notification",
        func=push,
        description="useful for when you want to send a push notification"
    )

In [5]:
# Introducing nest_asyncio
# Python async code only allows for one "event loop" processing aynchronous events.
# The `nest_asyncio` library patches this, and is used for special situations, if you need to run a nested event loop.

import nest_asyncio
nest_asyncio.apply()

In [6]:
from langchain_community.agent_toolkits import PlayWrightBrowserToolkit
from langchain_community.tools.playwright.utils import create_async_playwright_browser

# If you get a NotImplementedError here or later, see the Heads Up at the top of the notebook

async_browser =  create_async_playwright_browser(headless=False)  # headful mode
toolkit = PlayWrightBrowserToolkit.from_browser(async_browser=async_browser)
tools = toolkit.get_tools()

In [7]:
for tool in tools:
    print(f"{tool.name}={tool}")

click_element=async_browser=<Browser type=<BrowserType name=chromium executable_path=C:\Users\Owner\AppData\Local\ms-playwright\chromium-1194\chrome-win\chrome.exe> version=141.0.7390.37>
navigate_browser=async_browser=<Browser type=<BrowserType name=chromium executable_path=C:\Users\Owner\AppData\Local\ms-playwright\chromium-1194\chrome-win\chrome.exe> version=141.0.7390.37>
previous_webpage=async_browser=<Browser type=<BrowserType name=chromium executable_path=C:\Users\Owner\AppData\Local\ms-playwright\chromium-1194\chrome-win\chrome.exe> version=141.0.7390.37>
extract_text=async_browser=<Browser type=<BrowserType name=chromium executable_path=C:\Users\Owner\AppData\Local\ms-playwright\chromium-1194\chrome-win\chrome.exe> version=141.0.7390.37>
extract_hyperlinks=async_browser=<Browser type=<BrowserType name=chromium executable_path=C:\Users\Owner\AppData\Local\ms-playwright\chromium-1194\chrome-win\chrome.exe> version=141.0.7390.37>
get_elements=async_browser=<Browser type=<BrowserT

In [8]:
from langsmith import uuid7

tool_dict = {tool.name:tool for tool in tools}

navigate_tool = tool_dict.get("navigate_browser")
extract_text_tool = tool_dict.get("extract_text")

# Pass UUID v7 when running tools
await navigate_tool.arun(
    {"url": "https://www.cnn.com"},
    run_id=uuid7()
)
text = await extract_text_tool.arun(
    {},
    run_id=uuid7()
)